In [1]:
from stevetest_multiple_common import const, loader, predict

In [2]:
import pandas as pd
import sys
import json
import fastavro  # 0.17.9
import os
from multiprocessing import Pool
import logging

In [3]:
FORMAT = const.LOG_MSG_FORMAT
logging.basicConfig(format=FORMAT, datefmt=const.LOG_DATE_FORMAT)

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

model_lv1 = None
model_lv2 = None
model_lv3 = None

In [4]:
def load_model():
    global model_lv1
    model_lv1 = loader.load_model_lv1(const.get_model_file_path(const.FILE_PATH_MODEL_PRODUCTION) + "lv1/")
    
    global model_lv2s
    model_lv2s = loader.load_model_sub(const.get_model_file_path(const.FILE_PATH_MODEL_PRODUCTION) + "lv2/", 2)

    global model_lv3s
    model_lv3s = loader.load_model_sub(const.get_model_file_path(const.FILE_PATH_MODEL_PRODUCTION) + "lv3/", 3)

In [5]:
load_model()

[INFO] 2019-01-16 15:12:32 Loaded LEVEL1 model: cp_int_cate1_ROOT_100_0.5.bin
[INFO] 2019-01-16 15:12:32 Loading cp_int_cate2_56112_300_0.1.bin
[INFO] 2019-01-16 15:12:32 Loaded LEVEL2 model: cp_int_cate2_56112_300_0.1.bin
[INFO] 2019-01-16 15:12:32 Loading cp_int_cate2_69182_300_0.1.bin
[INFO] 2019-01-16 15:12:33 Loaded LEVEL2 model: cp_int_cate2_69182_300_0.1.bin
[INFO] 2019-01-16 15:12:33 Loading cp_int_cate2_80285_300_0.1.bin
[INFO] 2019-01-16 15:12:34 Loaded LEVEL2 model: cp_int_cate2_80285_300_0.1.bin
[INFO] 2019-01-16 15:12:34 Loading cp_int_cate3_56117_400_0.1.bin
[INFO] 2019-01-16 15:12:34 Loaded LEVEL3 model: cp_int_cate3_56117_400_0.1.bin
[INFO] 2019-01-16 15:12:34 Loading cp_int_cate3_69183_400_0.1.bin
[INFO] 2019-01-16 15:12:34 Loaded LEVEL3 model: cp_int_cate3_69183_400_0.1.bin
[INFO] 2019-01-16 15:12:34 Loading cp_int_cate3_80383_400_0.1.bin
[INFO] 2019-01-16 15:12:34 Loaded LEVEL3 model: cp_int_cate3_80383_400_0.1.bin


In [6]:
lst = list()
# read in one avro file
with open(const.get_cqi_input_file_path() + 'part-00000-ef5244bc-86ef-49a1-91e0-cc95a8f5cd95-c000.avro', 'rb') as fo:
    reader = fastavro.reader(fo)

    for record in reader:
        lst.append(
            [record['itemId'],
             record['productId'],
             record['categoryCode'],
             record['originalAttr'],
             record['normalizedAttr'],
             record['excludeType'],
             record['categoryCodeLv1'],
             record['categoryNameLv1']])

# noinspection PyUnresolvedReferences
df = pd.DataFrame(
    lst,
    columns=['itemId',
             'productId',
             'categoryCode',
             'originalAttr',
             'normalizedAttr',
             'excludeType',
             'categoryCodeLv1',
             'categoryNameLv1'])
df.head(100)

,itemId,productId,categoryCode,originalAttr,normalizedAttr,excludeType,categoryCodeLv1,categoryNameLv1
0,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,RETAIL,65799,반려/애완용품
1,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,RETAIL,65799,반려/애완용품
2,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,RETAIL,65799,반려/애완용품
3,9096629,11938,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,RETAIL,63897,생활용품
4,9096629,11938,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,RETAIL,63897,생활용품
5,9096629,11938,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,RETAIL,63897,생활용품
6,46485,12568,79134,Spice Hunter 인디아 샐러리 씨드 홀 Spice Hunter Season...,spice hunter 인디아 샐러리 씨드 홀 spice hunter season...,JIKGU,59258,식품
7,46485,12568,79134,Spice Hunter 인디아 샐러리 씨드 홀 Spice Hunter Season...,spice hunter 인디아 샐러리 씨드 홀 spice hunter season...,JIKGU,59258,식품
8,60289,13452,58633,Amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] Amy's 저지방 스프 A...,amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] amy's 저지방 스프 a...,JIKGU,59258,식품
9,60289,13452,58633,Amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] Amy's 저지방 스프 A...,amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] amy's 저지방 스프 a...,JIKGU,59258,식품


In [7]:
lst = None

df['originCateCode'] = df['categoryCode']
df['originString'] = df['originalAttr']
df['cleanseString'] = ''
df['predCateCode'] = ''
df['predCateCode1'] = ''
df['predCateCode2'] = ''
df['predCateCode3'] = ''
df['predCateCode4'] = ''
df['predCateCode5'] = ''
df['predCateCode6'] = ''
df['scoreCateCode1'] = 0.0
df['scoreCateCode2'] = 0.0
df['scoreCateCode3_6'] = 0.0
df['scoreFinal'] = 0.0
df['success'] = 0

# noinspection PyUnresolvedReferences
cleansed_prod_df = pd.read_csv(
    const.get_cleansed_prod_dictionary_file_name(),
    names=['productId', 'isCleansed'],
    sep='\t',
    dtype=[('productId', 'long'), ('isCleansed', 'str')])

# df = pd.merge(df, book_cate_df, on='originCateCode', how='left')
# df = pd.merge(df, jikgu_prod_df, on='productId', how='left')
# noinspection PyUnresolvedReferences
df = pd.merge(df, cleansed_prod_df, on='productId', how='left')
df = df.head(100)
df.head()

,itemId,productId,categoryCode,originalAttr,normalizedAttr,excludeType,categoryCodeLv1,categoryNameLv1,originCateCode,originString,...,predCateCode3,predCateCode4,predCateCode5,predCateCode6,scoreCateCode1,scoreCateCode2,scoreCateCode3_6,scoreFinal,success,isCleansed
0,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,RETAIL,65799,반려/애완용품,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,...,,,,,0.0,0.0,0.0,0.0,0,1
1,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,RETAIL,65799,반려/애완용품,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,...,,,,,0.0,0.0,0.0,0.0,0,1
2,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,RETAIL,65799,반려/애완용품,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,...,,,,,0.0,0.0,0.0,0.0,0,1
3,9096629,11938,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,RETAIL,63897,생활용품,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,...,,,,,0.0,0.0,0.0,0.0,0,NaN
4,9096629,11938,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,RETAIL,63897,생활용품,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,...,,,,,0.0,0.0,0.0,0.0,0,NaN


In [8]:
for i, row in df.iterrows():
    if not df.at[i, 'originString'] or len(df.at[i, 'originString']) == 0:
        continue

    pred = predict.predict(
        model_lv1,
        model_lv2s, model_lv3s,
        df.at[i, 'normalizedAttr'],  # input already garbage filtered string
        product_id=df.at[i, 'productId'],
        item_id=df.at[i, 'itemId'],
        garbage_filter=False)[0]
        
    df.at[i, 'cleanseString'] = pred.get_normalized_input_string()

    if "OLD" not in str(df.at[i, 'categoryNameLv1']).upper():

        if "JIKGU" in df.at[i, 'excludeType']:
            continue

        if "BOOK" in df.at[i, 'excludeType']:
            continue

        if "DVD" in df.at[i, 'excludeType']:
            continue

        if df.at[i, 'isCleansed'] == '1':
            if len(str(df.at[i, 'excludeType'])) == 0:
                df.at[i, 'excludeType'] = 'OPERATOR_MODEL'
            else:
                df.at[i, 'excludeType'] = str(df.at[i, 'excludeType']) + ',OPERATOR_MODEL'   
            continue

        if pred.get_predict_error() is True:
            continue

        if pred.get_final_score() < 0.25:
            df.at[i, 'scoreCateCode1'] = pred.get_lv1_score()
            df.at[i, 'scoreCateCode2'] = pred.get_lv2_score()
            df.at[i, 'scoreCateCode3_6'] = pred.get_lv3_score()
            df.at[i, 'scoreFinal'] = pred.get_final_score()
            continue

    df.at[i, 'predCateCode'] = pred.get_catecode()
    df.at[i, 'predCateCode1'] = pred.get_lv1_catecode()
    df.at[i, 'predCateCode2'] = pred.get_lv2_catecode()
    df.at[i, 'predCateCode3'] = pred.get_lv3_catecode()
    df.at[i, 'predCateCode4'] = pred.get_lv4_catecode()
    df.at[i, 'predCateCode5'] = pred.get_lv5_catecode()
    df.at[i, 'predCateCode6'] = pred.get_lv6_catecode()
    df.at[i, 'scoreCateCode1'] = pred.get_lv1_score()
    df.at[i, 'scoreCateCode2'] = pred.get_lv2_score()
    df.at[i, 'scoreCateCode3_6'] = pred.get_lv3_score()
    df.at[i, 'scoreFinal'] = pred.get_final_score()
    if pred.get_predict_error() is True:
        df.at[i, 'success'] = 0
    else:
        df.at[i, 'success'] = 1
            
df.head(100)

[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue1
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[INFO] 2019-01-16 15:12:45 continue4
[

11276 33307 ML 80285 0.5234 None 0.0 80383 1.0000 None 0.0 80285_80383_80384_80386__ 1.0000 None 0.0
11276 33307 ML 80285 0.5234 None 0.0 80383 1.0000 None 0.0 80285_80383_80384_80386__ 1.0000 None 0.0
11276 33307 ML 80285 0.5234 None 0.0 80383 1.0000 None 0.0 80285_80383_80384_80386__ 1.0000 None 0.0
11938 9096629 ML 69182 0.5508 None 0.0 69183 1.0000 None 0.0 69182_69183_69184_69188__ 0.7227 None 0.0
11938 9096629 ML 69182 0.5508 None 0.0 69183 1.0000 None 0.0 69182_69183_69184_69188__ 0.7227 None 0.0
11938 9096629 ML 69182 0.5508 None 0.0 69183 1.0000 None 0.0 69182_69183_69184_69188__ 0.7227 None 0.0
12568 46485 ML 69182 0.5918 None 0.0 69183 1.0000 None 0.0 69182_69183_69184_69187__ 0.7773 None 0.0
12568 46485 ML 69182 0.5918 None 0.0 69183 1.0000 None 0.0 69182_69183_69184_69187__ 0.7773 None 0.0
13452 60289 ML 69182 0.5313 None 0.0 69183 1.0000 None 0.0 69182_69183_69184_69187__ 0.9297 None 0.0
13452 60289 ML 69182 0.5313 None 0.0 69183 1.0000 None 0.0 69182_69183_69184_69187__ 

,itemId,productId,categoryCode,originalAttr,normalizedAttr,excludeType,categoryCodeLv1,categoryNameLv1,originCateCode,originString,...,predCateCode3,predCateCode4,predCateCode5,predCateCode6,scoreCateCode1,scoreCateCode2,scoreCateCode3_6,scoreFinal,success,isCleansed
0,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,"RETAIL,OPERATOR_MODEL",65799,반려/애완용품,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,...,,,,,0.0000,0.0,0.0000,0.0000,0,1
1,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,"RETAIL,OPERATOR_MODEL",65799,반려/애완용품,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,...,,,,,0.0000,0.0,0.0000,0.0000,0,1
2,33307,11276,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,"RETAIL,OPERATOR_MODEL",65799,반려/애완용품,65872,메라독 퓨어 구디스낵 애견간식 메라독 퓨어 구디스낵 애견간식 칠면조 + 감자 메라...,...,,,,,0.0000,0.0,0.0000,0.0000,0,1
3,9096629,11938,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,RETAIL,63897,생활용품,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,...,69184,69188,69188,69188,0.5508,1.0,0.7227,0.3981,1,NaN
4,9096629,11938,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,RETAIL,63897,생활용품,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,...,69184,69188,69188,69188,0.5508,1.0,0.7227,0.3981,1,NaN
5,9096629,11938,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,RETAIL,63897,생활용품,63901,녹스 자연이좋은 3겹데코 롤화장지 30m 녹스 자연이좋은 3겹데코 롤화장지 30m ...,...,69184,69188,69188,69188,0.5508,1.0,0.7227,0.3981,1,NaN
6,46485,12568,79134,Spice Hunter 인디아 샐러리 씨드 홀 Spice Hunter Season...,spice hunter 인디아 샐러리 씨드 홀 spice hunter season...,JIKGU,59258,식품,79134,Spice Hunter 인디아 샐러리 씨드 홀 Spice Hunter Season...,...,,,,,0.0000,0.0,0.0000,0.0000,0,1
7,46485,12568,79134,Spice Hunter 인디아 샐러리 씨드 홀 Spice Hunter Season...,spice hunter 인디아 샐러리 씨드 홀 spice hunter season...,JIKGU,59258,식품,79134,Spice Hunter 인디아 샐러리 씨드 홀 Spice Hunter Season...,...,,,,,0.0000,0.0,0.0000,0.0000,0,1
8,60289,13452,58633,Amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] Amy's 저지방 스프 A...,amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] amy's 저지방 스프 a...,JIKGU,59258,식품,58633,Amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] Amy's 저지방 스프 A...,...,,,,,0.0000,0.0,0.0000,0.0000,0,1
9,60289,13452,58633,Amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] Amy's 저지방 스프 A...,amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] amy's 저지방 스프 a...,JIKGU,59258,식품,58633,Amy's 저지방 스프 노 치킨 누들 베이스 [에이미스] Amy's 저지방 스프 A...,...,,,,,0.0000,0.0,0.0000,0.0000,0,1
